In [393]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [394]:
symbol = 'spy'
data = vbt.AlpacaData.download(symbol,
                               start='1990-3-05',
                               end='2023-3-16',
                               limit=1000000,
                               timeframe='1d').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'close': array([185.59, 183.69, 181.12, ..., 390.22, 387.78, 394.59]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([185.71, 185.87, 184.24, ..., 391.94, 387.99, 394.94]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([184.21, 183.43, 180.37, ..., 385.56, 382.23, 384.8 ]),
              'name': 'Candlestick',
              'open': array([184.5 , 185.53, 183.96, ..., 389.  , 384.41, 385.33]),
              'type': 'candlestick',
              'uid': '35e34c26-28f4-4928-b325-2a45020939a3',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 14, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 15, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 16, 4, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'yaxis': 'y'},
             {'marker': {'color': array(['#1b9e76', '#d95f02', '#d95f02', ..., '#1b9e76', '#1b9e76', '#1b9e76'],
                                        dtype=object),
                         'line': {'width': 0}},
              'name': 'Volume',
              'opacity': 0.5,
              'type': 'bar',
              'uid': 'f2f2fe19-5661-48a3-bd86-bba4fa27450f',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 14, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 15, 4, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 3, 16, 4, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x2',
              'y': array([9.78584180e+07, 1.08069059e+08, 1.66224154e+08, ..., 1.49772836e+08,
                          1.73294525e+08, 1.43753940e+08]),
              'yaxis': 'y2'}],
    'layout': {'bargap': 0,
               'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'showlegend': True,
               'template': '...',
               'width': 700,
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'matches': 'x2',
                         'rangeslider': {'visible': False},
                         'showgrid': True,
                         'showticklabels': False},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0], 'showgrid': True},
               'yaxis': {'anchor': 'x', 'domain': [0.3, 1.0], 'showgrid': True},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.3], 'showgrid': True}}
})

In [395]:
def pullback_place_entry(start, close, rsi_window = 10, sma_window = 200, entry = 30):
    rsi = vbt.RSI.run(close, rsi_window).rsi.to_numpy()
    sma = vbt.MA.run(close, sma_window).ma.to_numpy()


    entry = np.where((rsi < entry) & (sma < start), 1, 0)

    return entry, rsi, sma

PULLBACK_PLACE_ENTRY = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['start', 'close'],
    output_names = ['value', 'rsi', 'sma'],
    param_names=['rsi_window', 'sma_window', 'entry']
).from_apply_func(pullback_place_entry, rsi_window = 10, sma_window = 200, entry = 30)

enter = PULLBACK_PLACE_ENTRY.run(
    data['Open'],
    data['Close'],
    rsi_window = np.arange(10,70,step=2,dtype=int),
    sma_window = np.arange(50,300,step=10,dtype=int),
   # entry = np.arange(10,50,step=2,dtype=int),
    param_product = True
)

data['entry'] = enter.value
data['rsi'] = enter.rsi
data['sma'] = enter.sma

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

ValueError: Cannot set a DataFrame with multiple columns to the single column entry

In [ ]:
def pullback_place_exit(enter, rsi, out = 40, leave = 10):


    leave_array = np.roll(enter.astype(float), leave)
    leave_array[:leave] = np.inf
    #print(leave_array)

    stop_loss =  np.where((leave_array == 1), -1, 0)
    take_profit = np.where(rsi>out, -1, 0)
    sell = np.where((leave_array == 1) | (rsi>out) , -1, 0)

    return take_profit, stop_loss, sell

PULLBACK_PLACE_EXIT = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['enter','rsi'],
    output_names = ['take_profit', 'stop_loss', 'sell'],
    param_names=['rsi_window', 'out', 'leave']
).from_apply_func(pullback_place_exit, out = 40, leave=10)

exit = PULLBACK_PLACE_EXIT.run(
    enter.value,
    enter.rsi,
    #out= np.arange(10,70,step=2,dtype=int),
    #leave=np.arange(5,15,step=1,dtype=int),
    param_product = True
)

data['stop_loss'] = exit.stop_loss
data['take_profit'] = exit.take_profit
data['sell']=exit.sell


with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)


exits = exit.sell == -1
entries = enter.value == 1

entries = np.roll(entries, 2)
exits = np.roll(exits, 2)

pf = vbt.Portfolio.from_signals(data['Open'], entries, exits, init_cash=10000, freq='1d')

#orders = pf.orders

returns = pf.total_return()

print(returns.max())

fig = returns.vbt.heatmap(
        x_level = "comb_rsi_window",
        y_level = "comb_entry"
)
fig.show()
